 # Imports and Installs

In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

root_dir_path = os.getcwd()
root_dir_path = os.path.abspath(os.path.split(root_dir_path)[0])

In [2]:
from autogluon.common import space
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

# Model Training

In [ ]:
model_save_path = rf'{root_dir_path}/models/'

train_df = pd.read_csv(rf'{root_dir_path}/data/train_val_csv.csv', delimiter= '|', encoding='utf-8', index_col=0)

train_data = TabularDataset(train_df)
label = "KPB AKTIVITETSKOD"

train_data.head(5)

# Hyperparam Selection

In [15]:
"""hp_tune = True

nn_options = {
    'num_epochs': 10,
    'learning_rate': space.Real(1e-4, 1e-2, default=5e-4, log=True),
    'layers': space.Categorical([100],[1000],[200,100],[300,200,100]),
    'dropout_prob': space.Real(0.0, 0.5, default=0.1),
}

gbm_options = {
    'num_boost_round': 100,
    'num_leaves': space.Int(lower=26, upper=66, default=36),
}

hyperparameters = {
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,
                  }

time_limits = 2*60
num_trials = 5 
search_strategy = 'auto'

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
} """

In [ ]:
"""column_predictor = TabularPredictor(label=label, path=model_save_path).fit(train_data,
    time_limit=time_limits,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,)"""

In [ ]:
column_predictor = TabularPredictor(label=label, path=model_save_path).fit(train_data, presets='high_quality')

# Model Testing and Selection

In [20]:
val_data = TabularDataset(train_df)
val_data = val_data.sample(n=30000, random_state = 42)
val_data.drop(columns=['KPB AKTIVITETSKOD'])

val_predictor = TabularPredictor.load(model_save_path)

y_pred = val_predictor.predict(val_data)

In [ ]:
val_predictor.leaderboard(val_data)

In [1]:
cloned_model_save_path = rf'{root_dir_path}/models/'
val_predictor.clone_for_deployment()

NameError: name 'val_predictor' is not defined